In [2]:
import os 
import cv2

In [3]:
len(os.listdir(r'Image_2/train/train-org-img/'))

320

In [3]:
len(os.listdir(r'Image/train/train-label-img/'))

1445

In [4]:
train_image = []
for name in os.listdir(r'Image/train/train-org-img/'):
    train_image.append(name.split('.')[0])

In [5]:
train_label = []
for name in os.listdir(r'Image/train/train-label-img/'):
    train_label.append(name.split('_')[0])

In [6]:
# 交集
train_ = list(set(train_image) & set(train_label))
len(train_)

222

In [7]:
cv2.imread(r'Image/train/train-org-img/'+train_[0]+'.jpg')

array([[[36, 52, 41],
        [35, 51, 40],
        [19, 36, 25],
        ...,
        [32, 48, 37],
        [30, 44, 32],
        [28, 42, 30]],

       [[35, 51, 40],
        [38, 54, 43],
        [23, 40, 29],
        ...,
        [31, 47, 36],
        [32, 46, 34],
        [29, 43, 31]],

       [[37, 53, 42],
        [43, 59, 48],
        [35, 52, 41],
        ...,
        [34, 50, 39],
        [31, 48, 35],
        [27, 44, 31]],

       ...,

       [[26, 54, 41],
        [23, 51, 38],
        [28, 56, 43],
        ...,
        [38, 86, 80],
        [35, 83, 77],
        [32, 80, 74]],

       [[26, 54, 41],
        [23, 51, 38],
        [26, 54, 41],
        ...,
        [37, 85, 79],
        [34, 82, 76],
        [33, 81, 75]],

       [[26, 54, 41],
        [22, 50, 37],
        [23, 51, 38],
        ...,
        [32, 80, 74],
        [31, 79, 73],
        [34, 82, 76]]], dtype=uint8)

In [8]:
import os 
import cv2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Compose, ToTensor, Normalize, Resize

def create_df(image_path, label_path):

    train_image, train_label = [], []
    for name in os.listdir(image_path):
        train_image.append(name.split('.')[0])

    for name in os.listdir(label_path):
        train_label.append(name.split('_')[0])

    # 交集
    train_df = list(set(train_image) & set(train_label))
    return train_df


class image_dataset(Dataset):
    def __init__(self, image_path, label_path, x, mean, std) -> None:
        super().__init__()
        self.img_path = image_path
        self.label_path = label_path
        self.x = x
        self.mean = mean
        self.std = std
        self.transform_image = Compose([ToTensor(), Normalize(self.mean, self.std), Resize((128, 256))])
        self.transform_label = Compose([ToTensor(), Normalize(0, 1/255), Resize((128, 256))])

    def __len__(self, ):
        return len(self.x)

    def __getitem__(self, idx):
        img = cv2.imread(self.img_path + self.x[idx] + '.jpg')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        label = cv2.imread(self.label_path + self.x[idx] + '_lab.png', cv2.IMREAD_GRAYSCALE)  # cv2.IMREAD_GRAYSCALE np.ndaary()
        img = self.transform_image(img)
        label = self.transform_label(label).squeeze(0)
        return img, label
    

if __name__ == '__main__':

    df_list = create_df(image_path='Image/train/train-org-img/', label_path='Image/train/train-label-img/')
    print(df_list[0])
    train_set = image_dataset(image_path=r'Image/train/train-org-img/', label_path=r'Image/train/train-label-img/', x=df_list,
                              mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    train_loader = DataLoader(train_set, batch_size=16, shuffle=False)

    for i, (images, labels) in enumerate(train_loader):
        print(images.shape, labels.shape)


8305
torch.Size([16, 3, 128, 256]) torch.Size([16, 1, 128, 256])
torch.Size([16, 3, 128, 256]) torch.Size([16, 1, 128, 256])


KeyboardInterrupt: 